In [4]:
import webbrowser
import requests
import msal
import json

import os
os.chdir("/home/daniel/de_AI_Fabriek/belasting-agent-playground/")

from keys import O365_CLIENT_SECRET, O365_CLIENT_ID

# SCOPES = ["User.Read"]
SCOPES = ['Calendars.ReadWrite.Shared','offline_access']
GRAPH_API_ENDPOINT = 'https://graph.microsoft.com/v1.0'

# AUTHORITY = 'https://login.microsoftonline.com/' + "b4790366-ebc1-458b-8baf-4aed4d12f0fc"
AUTHORITY = 'https://login.microsoftonline.com/common'

# experimental

### general

In [93]:
import webbrowser
from datetime import datetime, timedelta
import json
import os
import msal


def generate_access_token(app_id, scopes):
    # Save Session Token as a token file
    access_token_cache = msal.SerializableTokenCache()

    # read the token file
    if os.path.exists('msg_api_token_access.json'):
        access_token_cache.deserialize(open("msg_api_token_access.json", "r").read())
        token_detail = json.load(open('msg_api_token_access.json',))
        token_detail_key = list(token_detail['RefreshToken'].keys())[0]
        modified = datetime.fromtimestamp(int(token_detail['RefreshToken'][token_detail_key]['last_modification_time']))
        end_date = modified + timedelta(days=90)
        if datetime.now() > end_date:
            os.remove('msg_api_token_access.json')
            access_token_cache = msal.SerializableTokenCache()

    # assign a SerializableTokenCache object to the client instance
    client = msal.PublicClientApplication(client_id=app_id, token_cache=access_token_cache)

    accounts = client.get_accounts()
    if accounts:
        # load the session
        token_response = client.acquire_token_silent(scopes, accounts[0])
    else:
        # authetnicate your accoutn as usual
        flow = client.initiate_device_flow(scopes=scopes)
        print('user_code: ' + flow['user_code'])
        webbrowser.open('https://microsoft.com/devicelogin')
        token_response = client.acquire_token_by_device_flow(flow)

    with open('msg_api_token_access.json', 'w') as _f:
        _f.write(access_token_cache.serialize())

    return token_response




## Calendar

In [ ]:
# TODO: Variable time zones

In [103]:
access_token = generate_access_token(O365_CLIENT_ID, SCOPES)
headers = {
    'Authorization': 'Bearer ' + access_token['access_token']
}

# create an event


### Create event

In [28]:
def construct_event_detail(event_name, **kwargs):
    request_body = {
        'subject': event_name,
    }
    for key, value in kwargs.items():
        request_body[key] = value
    return request_body


In [143]:
response1_create = requests.post(
    GRAPH_API_ENDPOINT + f'/me/events',
    headers=headers,
    json=construct_event_detail("test_event2")
)

print(json.dumps(response1_create.json(), indent=4))

{
    "@odata.context": "https://graph.microsoft.com/v1.0/$metadata#users('outlook_4C0C63D7D80EDD22%40outlook.com')/events/$entity",
    "@odata.etag": "W/\"lr+kWymW20KA+/WCcZNXrQAHzHElhA==\"",
    "id": "AQMkADAwATM3ZmYAZS1kYTE5LTMyMTQtMDACLTAwCgBGAAADXfncffxD_k_4msGd5-CKLAcAlr_kWymW20KA_-WCcZNXrQAAAgENAAAAlr_kWymW20KA_-WCcZNXrQAHzC34lwAAAA==",
    "createdDateTime": "2025-02-05T14:19:23.2808529Z",
    "lastModifiedDateTime": "2025-02-05T14:19:23.3517688Z",
    "changeKey": "lr+kWymW20KA+/WCcZNXrQAHzHElhA==",
    "categories": [],
    "transactionId": null,
    "originalStartTimeZone": "UTC",
    "originalEndTimeZone": "UTC",
    "iCalUId": "040000008200E00074C5B7101A82E00800000000EA494AF4D877DB01000000000000000010000000FD09D6ACF59FC8478EC6D8A0D70BF3DD",
    "uid": "040000008200E00074C5B7101A82E00800000000EA494AF4D877DB01000000000000000010000000FD09D6ACF59FC8478EC6D8A0D70BF3DD",
    "reminderMinutesBeforeStart": 15,
    "isReminderOn": true,
    "hasAttachments": false,
    "subject":

In [30]:
print(json.dumps(response1_create.json(), indent=4))


{
  "@odata.context": "https://graph.microsoft.com/v1.0/$metadata#users('outlook_4C0C63D7D80EDD22%40outlook.com')/events/$entity",
  "@odata.etag": "W/\"lr+kWymW20KA+/WCcZNXrQAHzHEdew==\"",
  "id": "AQMkADAwATM3ZmYAZS1kYTE5LTMyMTQtMDACLTAwCgBGAAADXfncffxD_k_4msGd5-CKLAcAlr_kWymW20KA_-WCcZNXrQAAAgENAAAAlr_kWymW20KA_-WCcZNXrQAHzC34jQAAAA==",
  "createdDateTime": "2025-02-05T11:38:41.7287303Z",
  "lastModifiedDateTime": "2025-02-05T11:38:41.7705395Z",
  "changeKey": "lr+kWymW20KA+/WCcZNXrQAHzHEdew==",
  "categories": [],
  "transactionId": null,
  "originalStartTimeZone": "UTC",
  "originalEndTimeZone": "UTC",
  "iCalUId": "040000008200E00074C5B7101A82E00800000000BF6E7B81C277DB010000000000000000100000003153086DC0AB3146AED38B48FB37B007",
  "uid": "040000008200E00074C5B7101A82E00800000000BF6E7B81C277DB010000000000000000100000003153086DC0AB3146AED38B48FB37B007",
  "reminderMinutesBeforeStart": 15,
  "isReminderOn": true,
  "hasAttachments": false,
  "subject": "test_event2",
  "bodyPreview":

In [ ]:
# TODO: figure out attendees

In [141]:
# Step 2.2 Example (Detailed)
event_name = 'Work Holiday'
body = {
    # html or text
    'contentType': 'html',
    'content': '<b>2 weeks vacation</b>'
}
start = {
    'dateTime': '2025-02-5T08:00:00',
    'timeZone': 'Europe/Amsterdam'
}
end = {
    'dateTime': '2025-02-17T17:00:00',
    'timeZone': 'Europe/Amsterdam'
}
location = {
    'displayName': 'Paris, France'
}
attendees = [
    {
        'emailAddress': {
            'address': 'daniel@vanoosteroom.com'
        },
         'type': 'required' # or optional
    }
]

In [142]:
response2_create = requests.post(
    GRAPH_API_ENDPOINT + f'/me/events',
    headers=headers,
    json=construct_event_detail(
            event_name,
            body=body,
            location=location,
            start=start,
            end=end,
            #attendees=attendees,
        )
)

In [105]:
print(json.dumps(response2_create.json(), indent=4))

{
    "@odata.context": "https://graph.microsoft.com/v1.0/$metadata#users('outlook_4C0C63D7D80EDD22%40outlook.com')/events/$entity",
    "@odata.etag": "W/\"lr+kWymW20KA+/WCcZNXrQAHzHElTg==\"",
    "id": "AQMkADAwATM3ZmYAZS1kYTE5LTMyMTQtMDACLTAwCgBGAAADXfncffxD_k_4msGd5-CKLAcAlr_kWymW20KA_-WCcZNXrQAAAgENAAAAlr_kWymW20KA_-WCcZNXrQAHzC34kgAAAA==",
    "createdDateTime": "2025-02-05T14:01:23.7191024Z",
    "lastModifiedDateTime": "2025-02-05T14:01:23.7809435Z",
    "changeKey": "lr+kWymW20KA+/WCcZNXrQAHzHElTg==",
    "categories": [],
    "transactionId": null,
    "originalStartTimeZone": "Europe/Amsterdam",
    "originalEndTimeZone": "Europe/Amsterdam",
    "iCalUId": "040000008200E00074C5B7101A82E008000000008208D370D677DB010000000000000000100000009B2AA678475CCA45B3956CBBC84AF3FC",
    "uid": "040000008200E00074C5B7101A82E008000000008208D370D677DB010000000000000000100000009B2AA678475CCA45B3956CBBC84AF3FC",
    "reminderMinutesBeforeStart": 15,
    "isReminderOn": true,
    "hasAttachmen

### Delete event

In [123]:
if "id" in response1_create.json():
    event_id1 = response1_create.json()['id']
    response1_delete = requests.delete(
        GRAPH_API_ENDPOINT + f'/me/events/{event_id1}',
        headers=headers
    )

In [124]:
if "id" in response2_create.json():
    event_id2 = response2_create.json()['id']
    response2_delete = requests.delete(
        GRAPH_API_ENDPOINT + f'/me/events/{event_id2}',
        headers=headers
    )

### get events

In [138]:
start_datetime = (datetime.now() - timedelta(hours=1)).isoformat()
end_datetime = (datetime.now() + timedelta(days=1)).isoformat()

In [144]:
response1_get = requests.get(
    GRAPH_API_ENDPOINT + f'/me/calendar/calendarView?startDateTime={start_datetime}&endDateTime={end_datetime}',
    headers=headers
)

In [145]:
response1_get.json()

{'@odata.context': "https://graph.microsoft.com/v1.0/$metadata#users('outlook_4C0C63D7D80EDD22%40outlook.com')/calendar/calendarView",
 'value': [{'@odata.etag': 'W/"lr+kWymW20KA+/WCcZNXrQAHzHElhA=="',
   'id': 'AQMkADAwATM3ZmYAZS1kYTE5LTMyMTQtMDACLTAwCgBGAAADXfncffxD_k_4msGd5-CKLAcAlr_kWymW20KA_-WCcZNXrQAAAgENAAAAlr_kWymW20KA_-WCcZNXrQAHzC34lwAAAA==',
   'createdDateTime': '2025-02-05T14:19:23.2808529Z',
   'lastModifiedDateTime': '2025-02-05T14:19:23.3517688Z',
   'changeKey': 'lr+kWymW20KA+/WCcZNXrQAHzHElhA==',
   'categories': [],
   'transactionId': None,
   'originalStartTimeZone': 'UTC',
   'originalEndTimeZone': 'UTC',
   'iCalUId': '040000008200E00074C5B7101A82E00800000000EA494AF4D877DB01000000000000000010000000FD09D6ACF59FC8478EC6D8A0D70BF3DD',
   'uid': '040000008200E00074C5B7101A82E00800000000EA494AF4D877DB01000000000000000010000000FD09D6ACF59FC8478EC6D8A0D70BF3DD',
   'reminderMinutesBeforeStart': 15,
   'isReminderOn': True,
   'hasAttachments': False,
   'subject': 'test

In [ ]:
{'@odata.context': "https://graph.microsoft.com/v1.0/$metadata#users('outlook_4C0C63D7D80EDD22%40outlook.com')/calendar/calendarView",
 'value': [
{'@odata.etag': 'W/"lr+kWymW20KA+/WCcZNXrQAHzHElhA=="',
   'id': 'AQMkADAwATM3ZmYAZS1kYTE5LTMyMTQtMDACLTAwCgBGAAADXfncffxD_k_4msGd5-CKLAcAlr_kWymW20KA_-WCcZNXrQAAAgENAAAAlr_kWymW20KA_-WCcZNXrQAHzC34lwAAAA==',
   'createdDateTime': '2025-02-05T14:19:23.2808529Z',
   'lastModifiedDateTime': '2025-02-05T14:19:23.3517688Z',
   'changeKey': 'lr+kWymW20KA+/WCcZNXrQAHzHElhA==',
   'categories': [],
   'transactionId': None,
   'originalStartTimeZone': 'UTC',
   'originalEndTimeZone': 'UTC',
   'iCalUId': '040000008200E00074C5B7101A82E00800000000EA494AF4D877DB01000000000000000010000000FD09D6ACF59FC8478EC6D8A0D70BF3DD',
   'uid': '040000008200E00074C5B7101A82E00800000000EA494AF4D877DB01000000000000000010000000FD09D6ACF59FC8478EC6D8A0D70BF3DD',
   'reminderMinutesBeforeStart': 15,
   'isReminderOn': True,
   'hasAttachments': False,
   'subject': 'test_event2',
   'bodyPreview': '',
   'importance': 'normal',
   'sensitivity': 'normal',
   'isAllDay': False,
   'isCancelled': False,
   'isOrganizer': True,
   'responseRequested': True,
   'seriesMasterId': None,
   'showAs': 'busy',
   'type': 'singleInstance',
   'webLink': 'https://outlook.live.com/owa/?itemid=AQMkADAwATM3ZmYAZS1kYTE5LTMyMTQtMDACLTAwCgBGAAADXfncffxD%2Bk%2B4msGd5%2FCKLAcAlr%2BkWymW20KA%2B%2FWCcZNXrQAAAgENAAAAlr%2BkWymW20KA%2B%2FWCcZNXrQAHzC34lwAAAA%3D%3D&exvsurl=1&path=/calendar/item',
   'onlineMeetingUrl': None,
   'isOnlineMeeting': False,
   'onlineMeetingProvider': 'unknown',
   'allowNewTimeProposals': True,
   'occurrenceId': None,
   'isDraft': False,
   'hideAttendees': False,
   'responseStatus': {'response': 'organizer', 'time': '0001-01-01T00:00:00Z'},
   'body': {'contentType': 'html',
    'content': '<html><head><meta http-equiv="Content-Type" content="text/html; charset=utf-8">\r\n<meta name="Generator" content="Microsoft Exchange Server">\r\n<!-- converted from text -->\r\n<style><!-- .EmailQuote { margin-left: 1pt; padding-left: 4pt; border-left: #800000 2px solid; } --></style></head>\r\n<body>\r\n<font size="2"><span style="font-size:11pt;"><div class="PlainText">&nbsp;</div></span></font>\r\n</body>\r\n</html>\r\n'},
   'start': {'dateTime': '2025-02-05T14:30:00.0002170', 'timeZone': 'UTC'},
   'end': {'dateTime': '2025-02-05T15:00:00.0002170', 'timeZone': 'UTC'},
   'location': {'displayName': '',
    'locationType': 'default',
    'uniqueIdType': 'unknown',
    'address': {},
    'coordinates': {}},
   'locations': [],
   'recurrence': None,
   'attendees': [],
   'organizer': {'emailAddress': {'name': 'Daniel van Oosteroom',
     'address': 'outlook_4C0C63D7D80EDD22@outlook.com'}},
   'onlineMeeting': None},
{'@odata.etag': 'W/"lr+kWymW20KA+/WCcZNXrQAHzHElgg=="',
   'id': 'AQMkADAwATM3ZmYAZS1kYTE5LTMyMTQtMDACLTAwCgBGAAADXfncffxD_k_4msGd5-CKLAcAlr_kWymW20KA_-WCcZNXrQAAAgENAAAAlr_kWymW20KA_-WCcZNXrQAHzC34lgAAAA==',
   'createdDateTime': '2025-02-05T14:19:18.0411682Z',
   'lastModifiedDateTime': '2025-02-05T14:19:18.0898548Z',
   'changeKey': 'lr+kWymW20KA+/WCcZNXrQAHzHElgg==',
   'categories': [],
   'transactionId': None,
   'originalStartTimeZone': 'Europe/Amsterdam',
   'originalEndTimeZone': 'Europe/Amsterdam',
   'iCalUId': '040000008200E00074C5B7101A82E0080000000077942BF1D877DB01000000000000000010000000C899C1D842DBC642A507A02C64FE9AFB',
   'uid': '040000008200E00074C5B7101A82E0080000000077942BF1D877DB01000000000000000010000000C899C1D842DBC642A507A02C64FE9AFB',
   'reminderMinutesBeforeStart': 15,
   'isReminderOn': True,
   'hasAttachments': False,
   'subject': 'Work Holiday',
   'bodyPreview': '2 weeks vacation',
   'importance': 'normal',
   'sensitivity': 'normal',
   'isAllDay': False,
   'isCancelled': False,
   'isOrganizer': True,
   'responseRequested': True,
   'seriesMasterId': None,
   'showAs': 'busy',
   'type': 'singleInstance',
   'webLink': 'https://outlook.live.com/owa/?itemid=AQMkADAwATM3ZmYAZS1kYTE5LTMyMTQtMDACLTAwCgBGAAADXfncffxD%2Bk%2B4msGd5%2FCKLAcAlr%2BkWymW20KA%2B%2FWCcZNXrQAAAgENAAAAlr%2BkWymW20KA%2B%2FWCcZNXrQAHzC34lgAAAA%3D%3D&exvsurl=1&path=/calendar/item',
   'onlineMeetingUrl': None,
   'isOnlineMeeting': False,
   'onlineMeetingProvider': 'unknown',
   'allowNewTimeProposals': True,
   'occurrenceId': None,
   'isDraft': False,
   'hideAttendees': False,
   'responseStatus': {'response': 'organizer', 'time': '0001-01-01T00:00:00Z'},
   'body': {'contentType': 'html',
    'content': '<html>\r\n<head>\r\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8">\r\n</head>\r\n<body>\r\n<b>2 weeks vacation</b>\r\n</body>\r\n</html>\r\n'},
   'start': {'dateTime': '2025-02-05T07:00:00.0000000', 'timeZone': 'UTC'},
   'end': {'dateTime': '2025-02-17T16:00:00.0000000', 'timeZone': 'UTC'},
   'location': {'displayName': 'Paris, France',
    'locationType': 'default',
    'uniqueId': 'Paris, France',
    'uniqueIdType': 'private'},
   'locations': [{'displayName': 'Paris, France',
     'locationType': 'default',
     'uniqueId': 'Paris, France',
     'uniqueIdType': 'private'}],
   'recurrence': None,
   'attendees': [],
   'organizer': {'emailAddress': {'name': 'Daniel van Oosteroom',
     'address': 'outlook_4C0C63D7D80EDD22@outlook.com'}},
   'onlineMeeting': None}]}

# Self build


In [91]:
def aquire_and_use_token(app_id, scopes, authority):
    access_token_cache = msal.SerializableTokenCache()
    
    # read token cache
    if os.path.exists("msg_api_token_access.json"):
        access_token_cache.deserialize(open("msg_api_token_access.json", "r").read())
        token_detail = json.load(open('msg_api_token_access.json',))
        token_detail_key = list(token_detail['RefreshToken'].keys())[0]
        modified = datetime.fromtimestamp(int(token_detail['RefreshToken'][token_detail_key]['last_modification_time']))
        end_date = modified + timedelta(days=90)
        if datetime.now() > end_date:
            os.remove('msg_api_token_access.json')
            access_token_cache = msal.SerializableTokenCache()

        
    client = msal.PublicClientApplication(
        app_id,
        authority=authority,
        token_cache=access_token_cache
    )
    
    token_response = None
     
    accounts = client.get_accounts()
                  
    if accounts:
        
        # Assuming the end user chose this one to proceed
        chosen_account = accounts[0]    
        
        # Now let's try to find a token in cache for this account
        # or by using refresh token
        token_response = client.acquire_token_silent(scopes, account=chosen_account)

    if not token_response:
        flow = client.initiate_device_flow(scopes=scopes)
        print('user code: ', flow['user_code'])
        webbrowser.open(flow['verification_uri'])
        
        token_response = client.acquire_token_by_device_flow(flow)
        print(flow["message"])
    
    if "access_token" in token_response:
        print("Token was obtained from:", token_response["token_source"])
        if os.getenv('ENDPOINT'):
            # Calling a web API using the access token
            api_result = requests.get(
                os.getenv('ENDPOINT'),
                hearders={'Authorization': 'Bearer ' + token_response["access_token"]},
            ).json()    
            print("Web API call result", json.dumps(api_result, indent=2))
        else:
            print("Token acquisition result", json.dumps(token_response, indent=2))
    else:
        print("Token acquisition failed", token_response)
    
    # write to persistent cache
    with open("msg_api_token_access.json", "w") as _f:
        _f.write(access_token_cache.serialize())
    return token_response
    

In [92]:
token_response = aquire_and_use_token(O365_CLIENT_ID, SCOPES, AUTHORITY)

Token was obtained from: cache
Token acquisition result {
  "access_token": "EwB4A8l6BAAUBKgm8k1UswUNwklmy2v7U/S+1fEAAYFjwKH/x5Ln6Fj6M2BkuT+/aZjtXFGhvuuru/H5vz+p2mCG+Bt+47XJ3+fUSeOxPD7XfIQW1VkRxCkPu+HfhHscjo7r9MzcZx9Ud33NRvH8yUkApb2+/U483idZ9XxDm4Bsi2+dADo07M7HLyMwyiFAX7YL55X1zTt8sKbGJbuDrnBdn+Swnls+6kOLY2pNf5dzjXzX+P48ehBOB/B7wSq9S1jt0VKoIu6dlfxE+6DTJo0W8K8IhLCV07IiU3N4VPgTmmIqSv2Gk3yquaA4t6L2P8RAVfES+WRSY/G0miJFf2/y63284GDmrgnNFKxhmRV5AbYWD18hwQIdm2jAdVUQZgAAEPv7GnrGhFkqfn2zKUPXn11AAo+MroxLNordIhTYogoRCg7+Q03P/cNUEKkY2RhhoPB3+rFZiGTii0ZVdFEscA0Fz34RY0lSRaqLuKUiLXkg5rwHNPZTJn/GiXixghvdS2AzihW1qSO+jQL2e/Leaqpdm1tk14q295k5W6YHaBLVFgnjQxivblh2c+a1eZlGYNiN/GiXGc6Z7T0tICaJoyhPfxTyfRhO6IUirzTBLzvC01q+UGHf6yc94+IvjS4F7YRvh2+S7cdnRaAbrtRXYdSfK+PuNDmP+amuEIRCV7elKP1NeTuVL4C6s2vQfeiIFUWQux3zbQCEUgRq2qniW/7ryeohMCdrAOSqldtXAUTpXkCM/Zdz95BCF/WcyR4pZHHxsKWKz6YKf0OsUrf4t6dxMrqIH2vP3y2MLyLqAYB6VGgXbq66BcfSW/UeA5gwihVPNK7uI5g+R6SPxM0X4JsmML9ao9+rw417zqd7/M46HvWMcnyQG1OpeZjf7KB1NXJJdCk4pbqTcFAiMpcvkad

In [5]:
token_response


{'token_type': 'Bearer',
 'scope': 'openid profile User.Read email',
 'expires_in': 4362,
 'ext_expires_in': 4362,
 'access_token': 'eyJ0eXAiOiJKV1QiLCJub25jZSI6ImlSR2ViNUdrV2FUS0wzbXJIM1hXSXl4clJJaHV2R3pfRkE3dWRNUXdzRzAiLCJhbGciOiJSUzI1NiIsIng1dCI6IllUY2VPNUlKeXlxUjZqekRTNWlBYnBlNDJKdyIsImtpZCI6IllUY2VPNUlKeXlxUjZqekRTNWlBYnBlNDJKdyJ9.eyJhdWQiOiIwMDAwMDAwMy0wMDAwLTAwMDAtYzAwMC0wMDAwMDAwMDAwMDAiLCJpc3MiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC8wMDMwZTJmOC05Njg4LTQ5YzgtYWU0Zi1jZWY2N2E3ZWM4ZmYvIiwiaWF0IjoxNzM4NzA4NTY4LCJuYmYiOjE3Mzg3MDg1NjgsImV4cCI6MTczODcxMzIzMSwiYWNjdCI6MCwiYWNyIjoiMSIsImFpbyI6IkFUUUF5LzhaQUFBQXVvMGVPQlhCVUtYSDh1WEhQak9hMDRVK2RhZm9yQ3JyS2hEYllzbTg3K3hrNmN3Wmc3bEVJcElmekdQbWc3dmUiLCJhbXIiOlsicHdkIl0sImFwcF9kaXNwbGF5bmFtZSI6ImJlbGFzdGluZy1hYW5naWZ0ZW4tYWdlbnQiLCJhcHBpZCI6IjZmZTRjOTlhLTc4ODYtNDQ2My05MWJiLWNmMTMzYzVkZDE4NCIsImFwcGlkYWNyIjoiMCIsImZhbWlseV9uYW1lIjoidmFuIE9vc3Rlcm9vbSIsImdpdmVuX25hbWUiOiJEYW5pZWwiLCJpZHR5cCI6InVzZXIiLCJpcGFkZHIiOiIyMDAxOjFjMDA6ZDA0OjIyMDA6ZmUyMDo4ND